In [1]:
from tasho.templates.P2P import P2P
from tasho import TaskModel, default_mpc_options
from tasho.templates.Regularization import Regularization
from tasho.Variable import Variable
from tasho.ConstraintExpression import ConstraintExpression
from tasho.Expression import Expression
from tasho.MPC import MPC 
from tasho import environment as env
from tasho import WorldSimulator
from tasho.OCPGenerator import OCPGenerator
from robotshyu import Robot as rob
from time import sleep
import numpy as np
import casadi as cs
import pybullet as p
import tasho
import robotshyu
import os

pybullet build time: May 20 2022 19:44:17


# P2P Redundant manipulator

In [2]:
robot = rob.Robot("kinova")
link_name = 7
goal_pose = Variable(robot.name, "goal_pose", "magic_number", (4,4), np.array(
        [[0, 1, 0, 0.6], [1, 0, 0, 0.0], [0, 0, -1, 0.25], [0, 0, 0, 1]]
    ))
q0 = [1.0193752249977548, -0.05311582280659044, -2.815452580946695,
    1.3191046402052224, 2.8582660722530533, 1.3988994390898029, 1.8226311094569714]
robot.set_joint_acceleration_limits(lb=-360*3.14159/180, ub=360*3.14159/180)
ndof = robot.nq
q_current = Variable(robot.name, "jointpos_init", 'magic_number', (ndof, 1), q0)

Loading robot params from json ...
Loaded 7-DoF robot: kinova


In [3]:
task_P2P = P2P(robot, link_name, goal_pose, q_current, 0.001)

In [4]:
# task_P2P.write_task_graph("P2P_Task.svg")

In [5]:
task_P2P.add_path_constraints(Regularization(task_P2P.variables['qd_'+robot.name], 1),
                              Regularization(task_P2P.variables['qdd_'+robot.name], 0.1))

In [6]:
# task_P2P.write_task_graph("P2P_Task2.svg")

In [7]:
horizon_steps = 10
horizon_period = 3.0
OCP_gen = OCPGenerator(task_P2P, False, {"time_period": horizon_period, "horizon_steps":horizon_steps})
q_ocp = OCP_gen.stage_tasks[0].variables['q_'+robot.name].x
OCP_gen.tc.set_initial(q_ocp, q0)
OCP_gen.tc.set_ocp_solver(
    "ipopt", 
    # {"ipopt":{"linear_solver":"ma27"}}
)
OCP_gen.tc.solve_ocp()


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.7, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:      529
Number of nonzeros in inequality constraint Jacobian.:      217
Number of nonzeros in Lagrangian Hessian.............:      168

Total number of variables............................:      224
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      164
Total number of inequality constrai

In [8]:
t_grid, qsol = OCP_gen.tc.sol_sample(q_ocp)
_, q_dot_sol = OCP_gen.tc.sol_sample(OCP_gen.stage_tasks[0].variables['qd_'+robot.name].x)

from tasho import WorldSimulator
import pybullet as p

obj = WorldSimulator.WorldSimulator()

    # Add robot to the world environment
position = [0.0, 0.0, 0.0]
orientation = [0.0, 0.0, 0.0, 1.0]
robotID = obj.add_robot(position, orientation, robot.name)
joint_indices = [0, 1, 2, 3, 4, 5, 6]

obj.resetJointState(robotID, joint_indices, q0)

for i in range(horizon_steps + 1):
    sleep(horizon_period*0.5/horizon_steps)
    obj.resetJointState(
        robotID, joint_indices, qsol[i]
    )

sleep(0.5)
obj.end_simulation()

Ending simulation


# Object picking MPC

In [9]:
robot = rob.Robot("kinova")
link_name = 7

# Define initial conditions of the robot
q0_val = [0, -0.523598, 0, 2.51799, 0, -0.523598, -1.5708]
qd0_val = [0] * robot.ndof
robot.set_joint_acceleration_limits(lb=-360*3.14159/180, ub=360*3.14159/180)
################################################
# Task spacification - Approximation to object
################################################

# Select prediction horizon and sample time for the MPC execution
horizon_size = 10
t_mpc = 0.05

q_init = Variable(robot.name, 'q_init', 'parameter', (robot.ndof,1))
qd_init = Variable(robot.name, 'qd_init', 'parameter', (robot.ndof,1))
goal_pose = Variable(robot.name, "goal_pose", 'parameter', (4,4))

task_P2P = P2P(robot, 7, goal_pose, q_init, 0.001)
# task_P2P.write_task_graph("P2P_Task4.svg")

Loading robot params from json ...
Loaded 7-DoF robot: kinova


In [10]:
task_P2P.remove_initial_constraints(task_P2P.constraint_expressions['stationary_qd_kinova'])
task_P2P.remove_terminal_constraints('rot_con_pose_7_kinova_vs_goal',
                                    'trans_con_pose_7_kinova_vs_goal')
# task_P2P.write_task_graph("P2P_Task5.svg")

In [11]:
task_P2P.add_initial_constraints(ConstraintExpression(robot.name, "eq", Expression(robot.name, "err_qd_qinit", lambda a, b : a - b, qd_init, task_P2P.variables['qd_kinova']),
                             "hard", reference = 0))
task_P2P.add_path_constraints(Regularization(task_P2P.expressions['trans_error_pose_7_kinova_vs_goal'], 1.0, norm = "L1"),
                            Regularization(task_P2P.expressions['ax_ang_error_pose_7_kinova_vs_goal'], 1), 
                            Regularization(task_P2P.variables['qd_kinova'], 1e-4),
                            Regularization(task_P2P.variables['qdd_kinova'], 1e-3))

# task_P2P.write_task_graph("P2P_Task6.svg")

In [12]:
pOCP = OCPGenerator(task_P2P, False, {"time_period":horizon_size*t_mpc, "horizon_steps":horizon_size})

tc = pOCP.tc

################################################
# Set solver and discretization options
################################################
mpc_options = default_mpc_options.get_default_mpc_options()

tc.ocp_solver = mpc_options["ipopt_init"]["solver_name"]
tc.ocp_options = mpc_options["ipopt_init"]["options"]
tc.mpc_solver = mpc_options["ipopt_lbfgs"]["solver_name"]
tc.mpc_options =mpc_options["ipopt_lbfgs"]["options"]
tc.set_ocp_solver(tc.ocp_solver, tc.ocp_options)

################################################
# Set parameter values
################################################
q = pOCP.stage_tasks[0].variables['q_kinova'].x
qd = pOCP.stage_tasks[0].variables['qd_kinova'].x
q_0 = pOCP.stage_tasks[0].variables['q_init_kinova'].x
qd_0 = pOCP.stage_tasks[0].variables['qd_init_kinova'].x
goal_pose = pOCP.stage_tasks[0].variables['goal_pose_kinova'].x
goal_pose_val =  cs.vertcat(
    cs.hcat([0, 1, 0, 0.5]),
    cs.hcat([1, 0, 0, 0.0]),
    cs.hcat([0, 0, -1, 0.25]),
    cs.hcat([0, 0, 0, 1]),
)
tc.set_initial(q, q0_val)
tc.set_value(goal_pose, goal_pose_val)
tc.set_value(q_0, q0_val)
tc.set_value(qd_0, qd0_val)

# Add an output port for joint velocities as well
tc.tc_dict["out_ports"].append({"name":"port_out_qd_kinova", "var":"qd_kinova", "desc": "output port for the joint velocities"})

# Add a monitor for termination criteria
tc.add_monitor({"name":"termination_criteria", "expression":cs.sqrt(cs.sumsqr(pOCP.stage_tasks[0].expressions["trans_error_pose_7_kinova_vs_goal"].x)) - 2e-2, "reference":0, "lower":True, "initial":True})

# os.system("export OMP_NUM_THREADS = 1")

sol = tc.solve_ocp()

startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa/X.org
GL_RENDERER=llvmpipe (LLVM 12.0.0, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 21.2.6
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 21.2.6
Vendor = Mesa/X.org
Renderer = llvmpipe (LLVM 12.0.0, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = Mesa/X.org
ven = Mesa/X.org
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,846]:

b3Printf: /home/mtplnr/mpc_ws/postech_lec/.venv/lib/python3.8/site-packages/robotshy

In [13]:
dir_casadi_func = "casadi_dir"
os.makedirs("./"+dir_casadi_func, exist_ok=True)
vars_db = tc.generate_MPC_component("./"+dir_casadi_func+"/", {"ocp_cg_opts":{"save":True, "codegen":False, "jit":False}, "mpc":True, "mpc_cg_opts":{"save":True, "codegen":False, "jit":False}})

@1=(opti1_p_2[12:15]-fk_T(opti1_x_1){7}[12:15]), (sqrt(dot(@1, @1))-0.02)
@1=(opti1_p_2[12:15]-fk_T(opti1_x_1){7}[12:15]), (sqrt(dot(@1, @1))-0.02)


In [14]:
MPC_component = MPC("kinova_obj_pickup", "./"+dir_casadi_func+"/"+ tc.name + ".json")

In [15]:
obj = WorldSimulator.WorldSimulator(bullet_gui=True)

# Add robot to the world environment
position = [0.0, 0.0, 0.0]
orientation = [0.0, 0.0, 0.0, 1.0]
kinovaID = obj.add_robot(position, orientation, "kinova")

# Set environment
environment = env.Environment()
package_path = tasho.__path__[0]
cube1 = env.Cube(length = 1, position = [0.5, -0.2, 0.35], orientation = [0.0, 0.0, 0.0, 1.0], urdf = package_path+"/models/objects/cube_small.urdf")
environment.add_object(cube1, "cube")
table1 = env.Box(height = 0.3, position = [0.5, 0, 0], orientation = [0.0, 0.0, 0.7071080798594737, 0.7071054825112364], urdf =package_path+ "/models/objects/table.urdf")
environment.add_object(table1, "table1")

cube2 = env.Cube(length = 1, position = [0.5, -0.2, 0.35], orientation = [0.0, 0.0, 0.0, 1.0], urdf = package_path+"/models/objects/cube_small_col.urdf", fixed = True)
environment.add_object(cube2, "cube2")
environment.set_in_world_simulator(obj)
cubeID = environment.get_object_ID("cube")
cube2ID = environment.get_object_ID("cube2")

p.resetBaseVelocity(cubeID, linearVelocity=[0, 0.8, 0])
# Determine number of samples that the simulation should be executed
no_samples = int(t_mpc / obj.physics_ts)
if no_samples != t_mpc / obj.physics_ts:
    print("[ERROR] MPC sampling time not integer multiple of physics sampling time")

# Correspondence between joint numbers in bullet and OCP
joint_indices = [0, 1, 2, 3, 4, 5, 6]

 # Begin the visualization by applying the initial control signal
ts, q_sol = tc.sol_sample(q, grid="control")
ts, q_dot_sol = tc.sol_sample(qd, grid="control")
obj.resetJointState(kinovaID, joint_indices, q0_val)
obj.setController(
    kinovaID, "velocity", joint_indices, targetVelocities=q_dot_sol[0]
)

In [ ]:
# Execute the MPC loop
q_log = []
q_dot_log = []
predicted_pos_log = []

for i in range(horizon_size * 100):
    print("----------- MPC execution -----------")

    q_now = obj.readJointPositions(kinovaID, joint_indices)
    qd_now = obj.readJointVelocities(kinovaID, joint_indices)
        
    MPC_component.input_ports["port_inp_q_init_kinova"]["val"] = q_now
    MPC_component.input_ports["port_inp_qd_init_kinova"]["val"] = qd_now

    # Predict the position of the target object (cube)
    lin_vel, ang_vel = p.getBaseVelocity(cubeID)
    lin_vel = cs.DM(lin_vel)
    lin_pos, _ = p.getBasePositionAndOrientation(cubeID)
    lin_pos = cs.DM(lin_pos)
    time_to_stop = cs.norm_1(lin_vel) / 0.3
    predicted_pos = (
        cs.DM(lin_pos)
        + cs.DM(lin_vel) * time_to_stop
        - 0.5 * 0.5 * lin_vel / (cs.norm_1(lin_vel) + 1e-3) * time_to_stop ** 2
    )
    predicted_pos_log.append(predicted_pos.full())
    p.resetBasePositionAndOrientation(cube2ID, predicted_pos.full(), [0.0, 0.0, 0.0, 1.0])
    predicted_pos[2] += 0.06  # cube height
    print("Predicted position of cube", predicted_pos)
    predicted_pos_val = cs.vertcat(
        cs.hcat([0, 1, 0, predicted_pos[0]]),
        cs.hcat([1, 0, 0, predicted_pos[1]]),
        cs.hcat([0, 0, -1, predicted_pos[2]]),
        cs.hcat([0, 0, 0, 1]),
    )

    MPC_component.input_ports["port_inp_goal_pose_kinova"]["val"] = cs.vec(predicted_pos_val)
        
    if i == 0:
        MPC_component.configMPC()

    MPC_component.runMPC()

    q_log.append(q_now)
    q_dot_log.append(qd_now)

    # Set control signal to the simulated robot
    qd_control_sig = MPC_component.output_ports["port_out_qd_kinova"]["val"].full()
    qdd_control_sig = (MPC_component.output_ports["port_out_qdd_kinova"]["val"] * t_mpc).full()
    obj.setController(
        kinovaID, "velocity", joint_indices, targetVelocities=qd_control_sig + qdd_control_sig
    )

    # Simulate
    obj.run_simulation(no_samples)

    # Termination criteria
#     if "termination_criteria_true" in MPC_component.event_output_port:
#         break

----------- MPC execution -----------
Predicted position of cube [0.5, 0.15777500346789, 0.41]
startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa/X.org
GL_RENDERER=llvmpipe (LLVM 12.0.0, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 21.2.6
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 21.2.6
Vendor = Mesa/X.org
Renderer = llvmpipe (LLVM 12.0.0, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = Mesa/X.org
ven = Mesa/X.org
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.

----------- MPC execution -----------
Predicted position of cube [0.4999996857523, 0.29580082187609, 0.2724997811708]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   1.01ms (100.80us) 474.26us ( 47.43us)        10
       nlp_g  |   4.32ms (432.30us)  86.24us (  8.62us)        10
    nlp_grad  | 133.00us (133.00us) 133.14us (133.14us)         1
  nlp_grad_f  |   2.04ms (185.09us)   1.36ms (123.59us)        11
  nlp_hess_l  |  28.82ms (  3.20ms)   6.42ms (713.00us)         9
   nlp_jac_g  | 166.00us ( 15.09us) 166.09us ( 15.10us)        11
       total  |  85.58ms ( 85.58ms)  13.76ms ( 13.76ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999968288122, 0.30652946718666, 0.2724933404809]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   1.26ms (114.55us) 535.43us ( 48.68us)        11
       nlp_g  | 149.00us ( 13.55us)  85.54us (  7.78us)        11
    nlp_grad  | 164.00us (164.00us) 1

----------- MPC execution -----------
Predicted position of cube [0.49999537645991, 0.42805661230223, 0.084991004298507]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 967.00us (120.88us) 404.02us ( 50.50us)         8
       nlp_g  |  64.00us (  8.00us)  63.40us (  7.92us)         8
    nlp_grad  |  24.14ms ( 24.14ms) 213.74us (213.74us)         1
  nlp_grad_f  |   1.80ms (200.44us)   1.22ms (136.05us)         9
  nlp_hess_l  |  20.93ms (  2.99ms)   5.54ms (791.67us)         7
   nlp_jac_g  | 146.00us ( 16.22us) 146.14us ( 16.24us)         9
       total  |  58.59ms ( 58.59ms)  11.59ms ( 11.59ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999530925173, 0.43080395082555, 0.084990010197714]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  16.69ms (  2.09ms) 397.53us ( 49.69us)         8
       nlp_g  |  69.00us (  8.62us)  67.16us (  8.40us)         8
    nlp_grad  | 189.00us (189.00

----------- MPC execution -----------
Predicted position of cube [0.49999362946391, 0.43634581441443, 0.084989789560905]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   1.13ms (188.67us) 281.35us ( 46.89us)         6
       nlp_g  |  47.00us (  7.83us)  49.01us (  8.17us)         6
    nlp_grad  |   1.46ms (  1.46ms) 165.06us (165.06us)         1
  nlp_grad_f  |   6.22ms (889.00us) 911.88us (130.27us)         7
  nlp_hess_l  |  17.84ms (  3.57ms)   4.04ms (808.41us)         5
   nlp_jac_g  | 116.00us ( 16.57us) 115.24us ( 16.46us)         7
       total  |  47.76ms ( 47.76ms)   8.62ms (  8.62ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999363075984, 0.4363456811995, 0.084989922476929]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 477.00us ( 79.50us) 288.46us ( 48.08us)         6
       nlp_g  |  50.00us (  8.33us)  50.46us (  8.41us)         6
    nlp_grad  | 163.00us (163.00u

----------- MPC execution -----------
Predicted position of cube [0.49999371678068, 0.43634552190005, 0.084989999999736]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   1.87ms (312.33us) 281.03us ( 46.84us)         6
       nlp_g  |  48.00us (  8.00us)  46.68us (  7.78us)         6
    nlp_grad  | 161.00us (161.00us) 161.34us (161.34us)         1
  nlp_grad_f  |   1.17ms (166.57us) 898.64us (128.38us)         7
  nlp_hess_l  |  13.83ms (  2.77ms)   3.92ms (784.23us)         5
   nlp_jac_g  | 112.00us ( 16.00us) 112.51us ( 16.07us)         7
       total  |  27.85ms ( 27.85ms)   8.39ms (  8.39ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999371678071, 0.43634552189993, 0.084989999999892]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 670.00us (111.67us) 286.94us ( 47.82us)         6
       nlp_g  | 133.00us ( 22.17us)  49.81us (  8.30us)         6
    nlp_grad  | 165.00us (165.00

----------- MPC execution -----------
Predicted position of cube [0.49999371678074, 0.43634552189998, 0.084989999999998]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 302.00us ( 50.33us) 302.97us ( 50.50us)         6
       nlp_g  |  50.00us (  8.33us)  48.56us (  8.09us)         6
    nlp_grad  | 186.00us (186.00us) 186.66us (186.66us)         1
  nlp_grad_f  |   9.06ms (  1.29ms) 926.49us (132.36us)         7
  nlp_hess_l  |  10.07ms (  2.01ms)   3.91ms (782.92us)         5
   nlp_jac_g  | 113.00us ( 16.14us) 114.59us ( 16.37us)         7
       total  |  34.79ms ( 34.79ms)   8.63ms (  8.63ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999371678074, 0.43634552189998, 0.084989999999998]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 278.00us ( 46.33us) 279.09us ( 46.52us)         6
       nlp_g  |  46.00us (  7.67us)  46.01us (  7.67us)         6
    nlp_grad  | 160.00us (160.00

----------- MPC execution -----------
Predicted position of cube [0.49999371678074, 0.43634552189998, 0.08499]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 363.00us ( 60.50us) 366.96us ( 61.16us)         6
       nlp_g  |  62.00us ( 10.33us)  61.80us ( 10.30us)         6
    nlp_grad  |   1.88ms (  1.88ms) 188.62us (188.62us)         1
  nlp_grad_f  |  15.63ms (  2.23ms)   1.15ms (163.92us)         7
  nlp_hess_l  |  12.65ms (  2.53ms)   4.77ms (953.21us)         5
   nlp_jac_g  | 468.00us ( 66.86us) 122.10us ( 17.44us)         7
       total  |  55.82ms ( 55.82ms)  10.46ms ( 10.46ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999371678074, 0.43634552189998, 0.08499]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 547.00us ( 91.17us) 403.43us ( 67.24us)         6
       nlp_g  | 123.00us ( 20.50us)  67.13us ( 11.19us)         6
    nlp_grad  |   1.97ms (  1.97ms) 129.71us (129.71

In [ ]:
obj.end_simulation()

max(MPC_component.solver_times)